In [ ]:
#IMPORTING LIBRARIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow

In [ ]:
#CLASSES
#0 - Speed limit (20km/h)  
#1 - Speed limit (30km/h)  
#2 - Speed limit (50km/h)  
#3 - Speed limit (60km/h)  
#4 - Speed limit (70km/h)  
#5 - Speed limit (80km/h)  
#6 - End of speed limit (80km/h)  
#7 - Speed limit (100km/h)  
#8 - Speed limit (120km/h)  
#9 - No passing  
#10 - No passing for vehicles over 3.5 metric tons  
#11 - Right-of-way at the next intersection  
#12 - Priority road  
#13 - Yield  
#14 - Stop  
#15 - No vehicles  
#16 - Vehicles over 3.5 metric tons prohibited  
#17 - No entry  
#18 - General caution  
#19 - Dangerous curve to the left  
#20 - Dangerous curve to the right  
#21 - Double curve  
#22 - Bumpy road  
#23 - Slippery road  
#24 - Road narrows on the right  
#25 - Road work  
#26 - Traffic signals  
#27 - Pedestrians  
#28 - Children crossing  
#29 - Bicycles crossing  
#30 - Beware of ice/snow  
#31 - Wild animals crossing  
#32 - End of all speed and passing limits  
#33 - Turn right ahead  
#34 - Turn left ahead  
#35 - Ahead only  
#36 - Go straight or right  
#37 - Go straight or left  
#38 - Keep right  
#39 - Keep left  
#40 - Roundabout mandatory  
#41 - End of no passing  
#42 - End of no passing by vehicles over 3.5 metric tons

In [ ]:
#IMPORTING THE DATASET

with open("./traffic-signs-data/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("./traffic-signs-data/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("./traffic-signs-data/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

In [ ]:
#CREATING TRAINING, VALIDATION AND TESTING DATA

X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
#PRINTING A MATRIX OF IMAGES  AND LABELS AT RANDOM

W_grid = 15
L_grid = 15
n_training = len(X_train)

fig, axes = plt.subplots(L_grid, W_grid, figsize=(20,20))
axes = axes.ravel()

for i in np.arange(0, L_grid * W_grid):
    index = np.random.randint(0,n_training)
    axes[i].imshow(X_train[index])
    axes[i].set_title(y_train[index], fontsize = 7)
    axes[i].axis('off')

In [ ]:
#CHECKING THE SHAPES OF THE DATA

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_validation shape: ", X_validation.shape)
print("y_validation shape: ", y_validation.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
#VISUALIZING THE DATA

i=1000
plt.imshow(X_train[i])
y_train[i]

In [ ]:
#SHUFFLING THE ORDER OF THE DATA

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
#GREYSCALING THE DATA

X_train_gray = np.sum(X_train/3, axis=3, keepdims=True)
X_validation_gray = np.sum(X_validation/3, axis=3, keepdims=True)
X_test_gray = np.sum(X_test/3, axis=3, keepdims=True)

In [ ]:
#CHECKING NEW SHAPE OF THE DATA

print("X_train_gray shape: ", X_train_gray.shape)
print("X_validation_gray shape: ", X_validation_gray.shape)
print("X_test_gray shape: ", X_test_gray.shape)

In [ ]:
#NORMALIZING THE DATA

X_train_gray_norm = (X_train_gray - 128)/128
X_validation_gray_norm = (X_validation_gray - 128)/128
X_test_gray_norm = (X_test_gray - 128)/128

In [ ]:
#VISUALIZING THE GRAYSCALED AND NORMALIZED DATA

i=610
plt.imshow(X_train[i])
plt.figure()
plt.imshow(X_train_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_train_gray_norm[i].squeeze(), cmap='gray')

In [ ]:
#BUILDING THE CNN - LE-NET DERIVED DEEP NETWORK

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

#Initializing
cnn = Sequential()

#First Convolutional Layer
cnn.add(Conv2D(filters = 6, kernel_size = (5,5), activation = 'relu', input_shape = (32,32,1)))

#Batch Normalization
cnn.add(BatchNormalization())

#First Pooling Layer
cnn.add(MaxPooling2D(pool_size=(2,2)))

#Second Convolutional Layer
cnn.add(Conv2D(filters = 16, kernel_size = (5,5), activation = 'relu'))

#Batch Normalization
cnn.add(BatchNormalization())

#Second Pooling Layer
cnn.add(MaxPooling2D(pool_size=(2,2)))

#Third Convolutional Layer
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))

#Batch Normalization
cnn.add(BatchNormalization())

#Second Pooling Layer
cnn.add(MaxPooling2D(pool_size=(2,2)))

#Flattening
cnn.add(Flatten())

#First Fully Connected Layer
cnn.add(Dense(units = 120, activation = 'relu'))

#Dropout Layer
cnn.add(Dropout(0.5))

#Second Fully Connected Layer
cnn.add(Dense(units = 84, activation = 'relu'))

#Dropout Layer
cnn.add(Dropout(0.5))

#Output Layer
cnn.add(Dense(units = 43, activation = 'softmax'))

In [ ]:
#COMPILING THE CNN

cnn.compile(optimizer = Adam(learning_rate=0.0003), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#ENHANCEMENTS - EARLY STOPPING

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True, min_delta=0.0005)
reduced_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=30, verbose=1, min_lr=1e-6)

In [ ]:
#ENHANCEMENTS - IMAGE AUGMENTATION
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range = 10,zoom_range = 0.1,width_shift_range = 0.1,height_shift_range = 0.1,horizontal_flip = True)
datagen.fit(X_train_gray_norm)

In [ ]:
#TRAINING THE CNN - DATA AUGMENTATION

history = cnn.fit(datagen.flow(X_train_gray_norm, y_train, batch_size=250),epochs=300, validation_data=(X_validation_gray_norm, y_validation), callbacks=[early_stopping,reduced_lr], verbose=1)

In [ ]:
#TRAINING THE CNN

#history = cnn.fit(X_train_gray_norm, y_train, batch_size = 250, epochs = 250, validation_data = (X_validation_gray_norm, y_validation), verbose = 1, callbacks=[early_stopping])

In [ ]:
#EVALUATING THE CNN

score = cnn.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy: {}'.format(score[1]))

In [ ]:
#CHECKING THE KEYS IN THE HISTORY

history.history.keys()

In [ ]:
#EVALUATING THE HISTORY

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
#VISUALIZING THE ACCURACY

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

#VISUALIZING THE LOSS
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
#PREDICTING THE TESTING DATA

predicted_classes = cnn.predict(X_test_gray_norm)
y_true = y_test

In [ ]:
#CONFUSION MATRIX

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, np.argmax(predicted_classes, axis=1))

In [ ]:
#PLOTTING THE CONFUSION MATRIX

plt.figure(figsize=(20, 15))
sns.heatmap(cm, annot=True, cmap='viridis', xticklabels=range(43), yticklabels=range(43), fmt='g')

In [ ]:
#PLOTTING PREDICTIONS TO TRUE VALUES
L = 7
W = 7
fig, axes = plt.subplots(L, W, figsize=(20, 15))
axes = axes.ravel()

for i in np.arange(0, L * W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(np.argmax(predicted_classes[i]), y_true[i]))
    axes[i].axis('off')

plt.subplots_adjust(wspace=1)
